In [1]:
import matplotlib
import argparse
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch
import torch.nn as nn # import modules

from utils.sampling import mnist_iid, mnist_noniid
from models_v4.Update import LocalTrain
from models_v4.Nets import MLP
from models_v4.Fed import FedAvg
from models_v4.test import test_acc

%load_ext autoreload
%autoreload 2
from torch.autograd import grad

from torch.autograd import grad

In [2]:
class my_argument:    
    epochs = 400    #"rounds of training"
    num_users = 100  # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep=1 #"the number of local epochs: E"
    local_bs=10 #"local batch size: B"
    bs=128 #"test batch size"
    lr=0.001 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    weight_decay = 5e-4
    opt = 'ADAM'

    # model arguments
    model = 'mlp'
    
    # other arguments
    dataset='mnist' #, help="name of dataset")
    iid=0
    num_classes=10#, help="number of classes")
    num_channels=1#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    
    seed=1#, help='random seed (default: 1)')
    model='mlp'
    q=20
    f_size=32
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cpu")
args.device = torch.device("cuda:3" if use_cuda else "cpu")
print(args.device)

True
cuda:3


In [3]:
from models_v4.Fed import FedAdd,FedSubstract,weight_vectorization_gen,FedAvg_gradient, weight_vectorization_gen2


In [4]:
def phiQ(p,sc, q, w):
    w_cap = w #[:,0]
    #w_cap=[item[0] if isinstance(item,list) and len(item)>0 else item for item in w_cap]
    #w_cap=np.array(w_cap)
    #print("w_cap_first")
    #print(w_cap)
    w_cap= sc*w_cap
    v=np.floor(q*w_cap)
    one=np.ones(len(w))
    r=np.random.uniform(0,1,len(w_cap))
    temp=(one.T+np.sign(q*w_cap-v-r))*np.sign(q*w_cap-v-r)
    #j=np.sign(w_cap-v-r)
    #print("jjj")
    #print(j)
    #temp= (1/q)*(1/2)*temp
    temp=(1/2)*temp
    #print("temp")
    #print(temp)
    #w_cap= (1/q)*v + temp
    w_cap=(1/q)*(v+temp)
    w_cap=q*w_cap
    #w_cap=w_cap+ (1/2)*p*(-np.sign(w_cap)+one.T)*(-np.sign(w_cap))
    w_cap=w_cap+ (1/2)*(p-5)*(-np.sign(w_cap)+one.T)*(-np.sign(w_cap))
    #print("w_cap_last")
    #print(w_cap)
    del temp
    del one
    del v
    del r
    return w_cap
args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

# load dataset and split users
if args.dataset == 'mnist':
    trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    dataset_train = datasets.MNIST('/data/mnist/', train=True, download=True, transform=trans_mnist)
    dataset_test = datasets.MNIST('/data/mnist/', train=False, download=True, transform=trans_mnist)
    # sample users
    if args.iid:
        dict_users = mnist_iid(dataset_train, args.num_users)
        print('iid dataset')
    else:
        dict_users = mnist_noniid(dataset_train, args.num_users)
        print("non iid dataset")

else:
    exit('Error: dataset not found')
img_size = dataset_train[0][0].shape


non iid dataset


/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:66: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [5]:
def toNumpyFlatArray(self):
        return self.flat


def updateFromNumpyFlatArray(self, arr):
    self.flat = arr
    start = 0
    new_glob = OrderedDict()
    for k in self.w_glob.keys():
        size = 1
        for dim in self.w_glob[k].shape:
            size *= dim
        shaped = np.reshape(arr[start : start + size].copy(), self.w_glob[k].shape)
        new_glob[k] = torch.from_numpy(shaped)
        start = start + size
    self.w_glob = new_glob
    self.net_glob.load_state_dict(self.w_glob)

In [6]:
# load dataset and split users
if args.dataset == 'mnist':
    trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    dataset_train = datasets.MNIST('./data/mnist/', train=True, download=True, transform=trans_mnist)
    dataset_test = datasets.MNIST('./data/mnist/', train=False, download=True, transform=trans_mnist)
    # sample users
    if args.iid:
        dict_users = mnist_iid(dataset_train, args.num_users)
    else:
        dict_users = mnist_noniid(dataset_train, args.num_users)
elif args.dataset == 'cifar':
    trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
    dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)
    if args.iid:
        print("iid dataset")
        dict_users = cifar_iid(dataset_train, args.num_users)
    else:
        exit('Error: only consider IID setting in CIFAR10')
else:
    exit('Error: unrecognized dataset')
img_size = dataset_train[0][0].shape

In [7]:
# beta=[1,1/2,1/4,1/8,1/16]
# #beta=[1,1,1,1,1]
# beta=[1,1,1/2,1/2,1/4]
# beta=[1,1/2,1/4,1/8]
# comp=[]
# i=0
# for j in range(args.num_users):
#     comp.append(beta[i])
#     if (j>0 and j%25==0):
#         i+=1
beta=[1,1/2,1/4,1/8,1/16]
beta=[1,1,1/2,1/2,1/4]
comp=[]
i=0
for j in range(args.num_users):
    comp.append(beta[i])
    if (j>0 and j%20==0):
        i+=1

In [8]:
print(comp)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]


In [9]:
#print(dict_users[2])

In [10]:
#print(dict_users[10])

In [11]:
print(len(dict_users[2]))

600


# 1. FedAvg with A=0

In [12]:

local_lr = 1e-4
local_steps = 1
use_updates = True

In [13]:
def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        seed=123
        torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform_(m.bias.data)
        torch.nn.init.zeros_(m.bias.data)


In [14]:
from models_v4.Fed import FedAdd,FedSubstract,weight_vectorization_gen,FedAvg_gradient
import numpy as np
import copy
import random
# build model

if args.model == 'mlp':
    len_in = 1
    for x in img_size:
        len_in *= x
    print(len_in)
    net_glob = MLP(dim_in=len_in, dim_hidden=200, dim_out=args.num_classes).to(args.device)
else:
    exit('Error:model not found')
print(net_glob)
net_glob.apply(weights_init)
net_glob.train()

784
MLP(
  (layer_input): Linear(in_features=784, out_features=200, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=200, out_features=10, bias=True)
)


MLP(
  (layer_input): Linear(in_features=784, out_features=200, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (layer_hidden): Linear(in_features=200, out_features=10, bias=True)
)

In [15]:
#print(w_glob['layer_hidden.weight'][:,s:s+y])

In [16]:
s_1=[]
for i in range(args.num_users):
    s_1.append(np.random.randint(200))

In [17]:
def update_local_model( w_hold,w_local, y,s):
    if (s+y>200):
            w_glob_hold['layer_input.weight'][0:y-(200-s),:]=w_local['layer_input.weight'][0:y-(200-s),:]
            w_glob_hold['layer_input.weight'][s:200,:]=w_local['layer_input.weight'][-(200-s):,:]
            #combined=torch.concatenate((first,middle),axis=0)
    
    else:
            w_glob_hold['layer_input.weight'][s:s+y,:]= w_local['layer_input.weight']
            #combined=middle
    #w_glob['layer_input.weight']=combined
    if (s+y>200):
            w_glob_hold['layer_input.bias'][0:y-(200-s)]=w_local['layer_input.bias'][0:y-(200-s)]
            w_glob_hold['layer_input.bias'][s:200]=w_local['layer_input.bias'][-(200-s):]
            #combined=torch.concatenate((first,middle),axis=0)
    
    else:
            w_glob_hold['layer_input.bias'][s:s+y] = w_local['layer_input.bias']
           
    #w_glob['layer_input.bias']=combined
        
    if (s+y>200):
            w_glob_hold['layer_hidden.weight'][:,0:y-(200-s)]=w_local['layer_hidden.weight'][:,0:y-(200-s)]
            w_glob_hold['layer_hidden.weight'][:,s:200]=w_local['layer_hidden.weight'][:,-(200-s):]
            #combined=torch.concatenate((first,middle),axis=1)
    else:
            w_glob_hold['layer_hidden.weight'][:,s:s+y]=w_local['layer_hidden.weight']
    w_glob_hold['layer_hidden.bias']=w_local['layer_hidden.bias']
            #combined=middle
    #w_glob['layer_hidden.weight']=combined
    return w_glob_hold

In [18]:
print(s_1)

[109, 126, 66, 98, 17, 83, 106, 123, 57, 96, 113, 126, 47, 73, 32, 174, 111, 153, 83, 78, 164, 96, 68, 49, 55, 195, 2, 84, 39, 66, 84, 47, 189, 176, 135, 105, 99, 124, 92, 180, 102, 97, 118, 94, 155, 34, 76, 168, 131, 106, 69, 64, 75, 162, 58, 138, 22, 146, 15, 155, 158, 180, 70, 109, 115, 154, 134, 14, 103, 182, 199, 129, 43, 186, 101, 183, 25, 178, 56, 49, 12, 18, 1, 51, 172, 117, 48, 56, 177, 86, 3, 67, 139, 149, 103, 98, 3, 139, 3, 6]


In [19]:
from models_v4.Fed import FedAdd,FedSubstract,weight_vectorization_cifar,FedAvg_gradient, weight_vectorization_gen, weight_vectorization_gen2
#net_glob = LeNet10().to(args.device)
#net_glob.train()
args.lr=0.0005
import torchvision.models as models
dev=torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

net_glob = MLP(dim_in=len_in, dim_hidden=200, dim_out=args.num_classes).to(args.device)
net_glob.apply(weights_init)
net_glob.train()
#net_glob.load_state_dict(w_glob)
net_glob=net_glob.to(dev)
net_glob.train()
# copy weights
#net_glob=torch.hub.load('pytorch/vision:v0.10.0','resnet18',pretrained=False)
#net_glob.eval()
w_glob = net_glob.state_dict()
g, dim = weight_vectorization_gen(w_glob)

w_glob=net_glob.state_dict()
# print(w_glob)
# print(w_glob.keys())
for k in w_glob.keys():
    print(w_glob[k].shape)
net_glob_original=copy.deepcopy(net_glob)
net_glob_original.to(dev)
w_glob_original=copy.deepcopy(w_glob)
w_glob_hold=net_glob_original.state_dict()
#print(w_glob_hold)
for h in w_glob_hold.keys():
    print(w_glob_hold[h].shape)
    #print(w_glob[h])
    w_glob_hold[h]=torch.zeros(w_glob_hold[h].shape)
    print(type(w_glob[h]))

torch.Size([200, 784])
torch.Size([200])
torch.Size([10, 200])
torch.Size([10])
torch.Size([200, 784])
<class 'torch.Tensor'>
torch.Size([200])
<class 'torch.Tensor'>
torch.Size([10, 200])
<class 'torch.Tensor'>
torch.Size([10])
<class 'torch.Tensor'>


In [20]:
#print(w_glob_original)

In [ ]:

# training
loss_train = []
loss_test_arr = []
acc_test_arr = []
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None
val_acc_list, net_list = [], []

m_local=[]
d=11699132 #resnet18
d= 11173962
d=11183582
d=21840
#d= 11699132
#d=62006
iter_no=2000
avg=[]
error=[]
idxs_users=range(0,args.num_users)
    #print(len(idxs_users))
user_no=args.num_users
updated=[]
a=[]
for user in idxs_users:
        #print(user)
    updated.append([])  
    a.append([])   
model_diff=[]




#net_glob.zero_grad()
input_gradient=[]

user_no=args.num_users

select=round(0.9*user_no)

K_local=round(1*d)
loss_train=[]    
#net_glob.zero_grad()   
for iter in range(iter_no): #args.epochs
    print("iteration no.",iter)
#     if (iter>=10):
#         args.lr=0.0001
    if (iter>=500):
        args.lr=args.lr /10
    m_local=[]
    f=[]
        #T=[]
    w_locals, loss_locals,diff_locals,grad_locals = [], [],[],[]
    m = 10
    updated=[]
    model_diff=[]
    grad_vect=[]
    prev=[]
    error=[]
    grad_vect_quant=[]
    grad_vect_quant2=[]
    grad_vect_send=[]
    grad_vect_send2=[]
    store_grad=[]
    location_local=[]
    rand=np.random.choice(idxs_users,user_no,replace=False)
    rand=idxs_users
    np.random.seed(iter)
    rand=np.random.choice(idxs_users, select, replace=False)
    rand=np.sort(rand)
    loss_train_user=[]
    for i in range(args.num_users):
        updated.append([])
        model_diff.append([])
        grad_vect.append([])
        prev.append([])
        grad_vect_send.append([])
        error.append(np.zeros(d))
        grad_vect_quant.append([])
    
    
    for user in rand : #rand: #idxs_users:
        #print(user)
        
        w_glob = net_glob_original.state_dict()
        
        s_1[user]=s_1[user]%200
        s=s_1[user]
#print(w_glob['layer_input.weight'][:,s:s+10])
        

        y=int(comp[user]*200)

        if (s+y>200):
            first=w_glob['layer_input.weight'][0:y-(200-s),:]
            #print(first.shape)
            middle=w_glob['layer_input.weight'][s:200,:]
            #print(middle.shape)
            combined=torch.concatenate((first,middle),axis=0)
            #print(combined.shape)
    
        else:
            middle=w_glob['layer_input.weight'][s:s+y,:]
            combined=middle
        w_glob['layer_input.weight']=combined
        if (s+y>200):
            first=w_glob['layer_input.bias'][0:y-(200-s)]
            middle=w_glob['layer_input.bias'][s:200]
            combined=torch.concatenate((first,middle),axis=0)
    
        else:
            middle=w_glob['layer_input.bias'][s:s+y]
            combined=middle
        w_glob['layer_input.bias']=combined
        
        if (s+y>200):
            first=w_glob['layer_hidden.weight'][:,0:y-(200-s)]
            middle=w_glob['layer_hidden.weight'][:,s:200]
            combined=torch.concatenate((first,middle),axis=1)
        else:
            middle=w_glob['layer_hidden.weight'][:,s:s+y]
            combined=middle
        w_glob['layer_hidden.weight']=combined
        
        #print(w_glob)
        
        net_glob = MLP(dim_in=len_in, dim_hidden=y, dim_out=args.num_classes).to(args.device)
        net_glob.train()
        net_glob.to(dev)
        
        net_glob.load_state_dict(w_glob)
        s_1[user]=(s_1[user]+1)%200
        
        prev[user]=copy.deepcopy(w_glob)
        local = LocalTrain(args=args, dataset=dataset_train, idxs=dict_users[user])
        w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
#         input_gradient=inversefed.reconstruction_algorithms.loss_steps(net_glob, ground_truth[user], labels[user], 
#                                                         lr=local_lr, local_steps=local_steps,
#                                                                    use_updates=use_updates)
        
        loss_train_user.append(loss)
    
        model_diff=FedSubstract(w,prev[user])
        #print(model_diff)
        
#         g, dim = weight_vectorization_gen(model_diff)
#         print(len(g))
        
        model_diff=update_local_model(w_glob_hold, model_diff, y,s)
        #print(model_diff2)
        
        g, dim = weight_vectorization_gen(model_diff)
        #print(len(g))
        g=g[:,0] #+error[user]
        grad1=abs(np.array(g))
        
        grad_vect[user]= g #np.multiply(g,mask)
        scale=1/(1*0.9*args.num_users)
        grad_vect_quant[user]= scale*grad_vect[user] #phiQ(np.power(2,args.f_size),scale,2**args.q,grad_vect[user])
        #error[user]=g-grad_vect[user]
        grad_locals.append(grad_vect_quant[user])
        del g
        #print(user)
    grad_avg=sum(grad_locals) #/len(grad_locals)
    loss_train.append(sum(loss_train_user)/len(loss_train_user))
    grad_avg=sum(grad_locals) # taking the average of masked gradients
    
    grad_avg_correct = grad_avg #np.zeros_like(grad_avg)
    
    p=np.power(2,args.f_size)-5

    count=0
    w_glob_prev=copy.deepcopy(w_glob_original)
    flat=[]
    #conver
    for i in range(len(w_glob.keys())): # 4 layers in parameter
        flat.append([])

    for h in w_glob_prev.keys():
        s=list(w_glob_original[h].shape)
        if (len(s)==0):
            new=np.array(0)
            grad_avg_correct=np.delete(grad_avg_correct,np.s_[0])
        else:
            z=np.prod(list(w_glob_original[h].shape))
            flat[count]=grad_avg_correct[0:z] # taking out the vector for the specified layer
            grad_avg_correct=np.delete(grad_avg_correct,np.s_[0:z]) # deleting that vector from decoded after taking out
             
            new=flat[count].reshape(list(w_glob_original[h].shape)) #reshaping back to the marix
              
        w_glob_original[h]=torch.from_numpy(new) #converting the matrix to a tensor
            #print(w_glob[cluster_no][h].shape)
        count=count+1
    global_diff = w_glob_original
    #print(w_glob)
    w_glob_original=FedAdd(w_glob_prev,global_diff)
    # update global weights
    #global_diff = w_glob
    #print(w_glob)
    #w_glob=FedAdd(w_glob_prev,global_diff)
    

    # copy weight to net_glob
    net_glob_original.load_state_dict(w_glob_original)
    
    del w_glob_prev
    del grad_locals
    del grad_avg
    del flat
    torch.cuda.empty_cache()

    # print loss
    #loss_avg = np.nan_to_num(sum(loss_locals) / len(loss_locals))
    
    #loss_train.append(float(loss_avg))
    
    acc_test, loss_test = test_acc(net_glob, dataset_test, args)
    acc_test_arr.append(float(acc_test))
    loss_test_arr.append(loss_test)
    if iter % 1 ==0:
        #print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
 
        print("accuracy array")
        print(acc_test_arr[iter])
        print("train loss")
        print(loss_train[iter])       
        
        
   
       



iteration no. 0
accuracy array
9.5600004196167
train loss
0.4728033218823663
iteration no. 1
accuracy array
24.3799991607666
train loss
0.3634827059504733
iteration no. 2
accuracy array
31.809999465942383
train loss
0.3067583966095929
iteration no. 3
accuracy array
40.7599983215332
train loss
0.27722716654862967
iteration no. 4
accuracy array
45.22999954223633
train loss
0.2496264646833257
iteration no. 5
accuracy array
48.83000183105469
train loss
0.23446507055779225
iteration no. 6
accuracy array
56.02000045776367
train loss
0.22515978208831086
iteration no. 7
accuracy array
56.209999084472656
train loss
0.20129812622766016
iteration no. 8
accuracy array
57.150001525878906
train loss
0.20218974664412864
iteration no. 9
accuracy array
54.91999816894531
train loss
0.19640451623996608
iteration no. 10
accuracy array
58.7599983215332
train loss
0.19595456757933924
iteration no. 11
accuracy array
60.04999923706055
train loss
0.19239409458913548
iteration no. 12
accuracy array
65.5
train l

In [ ]:
print(model_diff['layer_hidden.weight'])
#print(model_diff2['layer_hidden.weight'])

In [ ]:
for k in w_glob.keys():
    print(w_glob[k].shape)

In [ ]:
print(grad_avg_correct)

In [ ]:
print(w_glob_original)

In [ ]:
for k in model_diff.keys():
    print(model_diff[k].shape)

In [ ]:
print(s_1[user])

In [ ]:
print(s_1)

In [ ]:
import matplotlib
step=1000
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})
matplotlib.rc('xtick', labelsize=18) 
matplotlib.rc('ytick', labelsize=18) 
#plt.plot(range(len(Cluster0_minor1)), Cluster0_minor1,label="users=2,5,5,6,7")
plt.plot(range(len(acc_test_arr[0:step])), acc_test_arr[0:step],label="without sparsification")
plt.ylabel('Test accuracy')
plt.xlabel('# Global Rounds')
#plt.legend()
#plt.figure(figsize=(6,5), dpi=400)
plt.show()

In [ ]:
print(max(acc_test_arr[0:1000]))

In [ ]:
#print(loss_train)

In [ ]:
print(acc_test_arr)

In [ ]:
#torch.save(net_glob.state_dict(),"model_cnn.pt")
#for k in w_glob.keys():
#print(w_glob['conv1.weight'].to(torch.device("cuda"))-w_glob['conv1.weight'].to(torch.device("cuda")))

In [ ]:
print(loss_train)